# Thư viện cần thiết

In [10]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold, RandomizedSearchCV
import pandas as pd
import numpy as np

# Load dữ liệu

In [11]:
X_train = pd.read_csv('../exps/feature_Labels/ex1_train.csv')
X_test = pd.read_csv('../exps/feature_Labels/ex1_test.csv')
temp = pd.read_csv('../data/train.csv')
y = y = np.log(temp["SalePrice"])

# Tối ưu hóa, Huấn luyện và test

In [12]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)

In [13]:
def random_search(model, grid, n_iter=100):
    search = RandomizedSearchCV(estimator = model, param_distributions = grid, cv=kf, n_iter = n_iter, n_jobs=4, random_state=0, verbose=True)
    return search.fit(X_train, y)

In [14]:
svr_hpg = {"C": np.arange(1, 100), "gamma": np.linspace(0.00001, 0.001, 50), "epsilon": np.linspace(0.01, 0.1, 50)} #Support Vector Regressor
gbm_hpg = {"max_features": np.linspace(0.2, 0.7, 6), "learning_rate": np.logspace(-3, -1, 100)} #Gradient Boost
rfr_hpg = {"n_estimators": [100, 300, 500, 1000], "max_features": np.linspace(0.1, 0.5, 5),"max_depth": [10, 20, 30, None], "min_samples_split": [2, 5, 10],"min_samples_leaf": [1, 2, 4]}

In [15]:
vr_search = random_search(SVR(), svr_hpg,n_iter=100) #Support Vector Regressor
rfr_search = random_search(RandomForestRegressor(),rfr_hpg,n_iter=100) #Random Forest Regression
gbm_search = random_search(GradientBoostingRegressor(n_estimators=2000, max_depth=3),gbm_hpg,n_iter=100) #Gradient Boost

ValueError: Found input variables with inconsistent numbers of samples: [1440, 1460]